* Models we have learnt
* Classification-> Target: Categorical or binary
  * DummyClassifier
  * DecisionTree
  * KNN
  * Naive Bayers -> Text content extraction
  * Logistics Regression
* Prediction-> Target: Numeric, continous
  * DummyPrediction
  * Linear Regression
  * *Non-Linear Regression (Not Covered)*
* Preprocessing:
  * imputering->fillna
  * encode:
    * ordinal
    * one-hot
  * scalaring-> for SVM like model
  * standarlizing-> for SVM like model
  * column transformation
  * make_pipline
* Hyperparameter Optimazation
  * validation check
  * grid/random search
* Final:
  * data visualization:
    * descriptive analysis
    * decision boundaries
    * tree map
    * validation/train score
  * report delivery

In [1]:
# create sql connection
from sqlalchemy import create_engine
import pandas as pd
import json
import urllib.parse
import psycopg2
import os 
with open("credentials.json") as file:
    login = json.load(file)
    
username = login['user']
password = urllib.parse.quote(login['password'])
host = login['host']
port = login['port']

engine = create_engine("postgresql://{}:{}@{}:{}/postgres".format(username,password,host,port))